# 02. LangGraph State & Memory - Quản lý Trạng thái và Bộ nhớ

## Mục tiêu học tập

- Hiểu vai trò quan trọng của **State** trong LangGraph
- Nắm vững cách tạo và quản lý trạng thái tùy chỉnh
- Học cách các Node đọc và cập nhật State
- Khám phá khả năng lưu trữ bộ nhớ lâu dài (Persistent Memory)
- Thực hành với các pattern State management phổ biến

## Giới thiệu về State trong LangGraph

**State** là trái tim của LangGraph - nó cho phép:

1. **Lưu trữ ngữ cảnh**: Duy trì thông tin qua các bước thực thi
2. **Truyền dữ liệu**: Chia sẻ thông tin giữa các Node
3. **Theo dõi tiến trình**: Ghi nhận trạng thái thực thi của workflow
4. **Điều khiển luồng**: Quyết định đường đi dựa trên trạng thái hiện tại

### Các khái niệm cơ bản:

- **StateGraph**: Graph có khả năng quản lý trạng thái
- **State Schema**: Định nghĩa cấu trúc dữ liệu của trạng thái
- **State Update**: Cách thức cập nhật trạng thái trong Node
- **Persistent Memory**: Lưu trữ trạng thái lâu dài

## Cài đặt & Cấu hình

In [ ]:
# Cài đặt các thư viện cần thiết
!pip install langgraph langchain-openai python-dotenv typing-extensions

In [ ]:
import os
from dotenv import load_dotenv
from typing import TypedDict, List, Optional, Any
from datetime import datetime
import uuid

from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI

# Load environment variables
load_dotenv()

# Thiết lập OpenAI API
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    print("⚠️ Cần thiết lập OPENAI_API_KEY trong file .env")
else:
    print("✅ OpenAI API Key đã được thiết lập")

## Ví dụ 1: Tạo StateGraph với Fields tùy chỉnh

Đây là ví dụ cơ bản về cách định nghĩa State Schema cho một chatbot đơn giản.

In [ ]:
# Định nghĩa State Schema
class ChatState(TypedDict):
    """Schema định nghĩa cấu trúc trạng thái cho chatbot"""
    messages: List[str]  # Lịch sử tin nhắn
    user_name: str       # Tên người dùng
    conversation_id: str # ID cuộc hội thoại
    timestamp: str       # Thời gian cuối cùng cập nhật
    context: dict        # Ngữ cảnh bổ sung

# Khởi tạo LLM
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.7,
    api_key=openai_api_key
)

print("✅ Đã định nghĩa ChatState và khởi tạo LLM")

## Ví dụ 2: Node đọc & cập nhật trạng thái

Các Node trong LangGraph có thể đọc và cập nhật State một cách linh hoạt.

In [ ]:
def initialize_conversation(state: ChatState) -> ChatState:
    """Node khởi tạo cuộc hội thoại"""
    print(f"🚀 Khởi tạo cuộc hội thoại cho {state.get('user_name', 'Guest')}")
    
    # Cập nhật state với thông tin khởi tạo
    state["conversation_id"] = str(uuid.uuid4())[:8]
    state["timestamp"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    state["messages"] = state.get("messages", [])
    state["context"] = state.get("context", {})
    
    # Thêm tin nhắn chào mừng
    welcome_msg = f"Xin chào {state['user_name']}! Tôi là AI Assistant của bạn."
    state["messages"].append(f"Bot: {welcome_msg}")
    
    print(f"📝 Conversation ID: {state['conversation_id']}")
    return state

def process_user_input(state: ChatState) -> ChatState:
    """Node xử lý input từ người dùng"""
    messages = state.get("messages", [])
    
    # Lấy tin nhắn cuối từ user (nếu có)
    user_messages = [msg for msg in messages if msg.startswith("User:")]
    
    if user_messages:
        last_user_msg = user_messages[-1].replace("User: ", "")
        print(f"👤 Xử lý tin nhắn: {last_user_msg}")
        
        # Cập nhật context dựa trên nội dung tin nhắn
        if "tên" in last_user_msg.lower():
            state["context"]["asked_about_name"] = True
        if "thời tiết" in last_user_msg.lower():
            state["context"]["asked_about_weather"] = True
    
    # Cập nhật timestamp
    state["timestamp"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    return state

def generate_response(state: ChatState) -> ChatState:
    """Node tạo phản hồi từ AI"""
    messages = state.get("messages", [])
    context = state.get("context", {})
    user_name = state.get("user_name", "bạn")
    
    # Lấy tin nhắn user cuối cùng
    user_messages = [msg for msg in messages if msg.startswith("User:")]
    
    if user_messages:
        last_user_msg = user_messages[-1].replace("User: ", "")
        
        # Tạo prompt có ngữ cảnh
        context_info = ""
        if context.get("asked_about_name"):
            context_info += f"Người dùng tên {user_name}. "
        
        prompt = f"""
        Bạn là một AI Assistant thân thiện. {context_info}
        Người dùng vừa nói: "{last_user_msg}"
        Hãy trả lời một cách tự nhiên và hữu ích.
        """
        
        # Gọi LLM để tạo phản hồi
        try:
            response = llm.invoke(prompt)
            ai_response = response.content
            print(f"🤖 AI phản hồi: {ai_response}")
        except Exception as e:
            ai_response = f"Xin lỗi {user_name}, tôi gặp lỗi khi xử lý tin nhắn của bạn."
            print(f"❌ Lỗi LLM: {e}")
        
        # Thêm phản hồi vào messages
        state["messages"].append(f"Bot: {ai_response}")
    
    return state

print("✅ Đã định nghĩa các Node xử lý State")

## Ví dụ 3: Truyền trạng thái qua các bước

Xây dựng workflow hoàn chỉnh với StateGraph.

In [ ]:
# Tạo StateGraph
workflow = StateGraph(ChatState)

# Thêm các nodes
workflow.add_node("initialize", initialize_conversation)
workflow.add_node("process_input", process_user_input)
workflow.add_node("generate_response", generate_response)

# Định nghĩa edges (luồng thực thi)
workflow.add_edge(START, "initialize")
workflow.add_edge("initialize", "process_input")
workflow.add_edge("process_input", "generate_response")
workflow.add_edge("generate_response", END)

# Compile graph
chatbot_graph = workflow.compile()

print("✅ Đã tạo và compile StateGraph")

In [ ]:
# Test workflow với input đầu vào
def test_chatbot_workflow():
    """Test workflow với một cuộc hội thoại mẫu"""
    
    print("=== KIỂM TRA WORKFLOW CHATBOT ===")
    
    # Khởi tạo state ban đầu
    initial_state = {
        "user_name": "Minh",
        "messages": ["User: Xin chào! Tôi tên Minh."],
        "conversation_id": "",
        "timestamp": "",
        "context": {}
    }
    
    print("\n🔍 State ban đầu:")
    for key, value in initial_state.items():
        print(f"  {key}: {value}")
    
    # Chạy workflow
    print("\n🔄 Chạy workflow...")
    result = chatbot_graph.invoke(initial_state)
    
    print("\n✅ State sau khi thực thi:")
    for key, value in result.items():
        if key == "messages":
            print(f"  {key}:")
            for i, msg in enumerate(value, 1):
                print(f"    {i}. {msg}")
        else:
            print(f"  {key}: {value}")
    
    return result

# Chạy test
final_state = test_chatbot_workflow()

## Ví dụ 4: Persistent Memory với MemorySaver

LangGraph hỗ trợ lưu trữ trạng thái lâu dài thông qua Checkpointer.

In [ ]:
# Tạo MemorySaver để lưu trữ trạng thái
memory_saver = MemorySaver()

# Tạo StateGraph với persistent memory
persistent_workflow = StateGraph(ChatState)

# Thêm các nodes (tái sử dụng từ trước)
persistent_workflow.add_node("initialize", initialize_conversation)
persistent_workflow.add_node("process_input", process_user_input)
persistent_workflow.add_node("generate_response", generate_response)

# Định nghĩa edges
persistent_workflow.add_edge(START, "initialize")
persistent_workflow.add_edge("initialize", "process_input")
persistent_workflow.add_edge("process_input", "generate_response")
persistent_workflow.add_edge("generate_response", END)

# Compile với memory saver
persistent_chatbot = persistent_workflow.compile(checkpointer=memory_saver)

print("✅ Đã tạo Persistent Chatbot với MemorySaver")

In [ ]:
# Tạo hàm để chat liên tục với persistent memory
def chat_with_memory(user_input: str, thread_id: str = "user_123"):
    """Chat với chatbot có bộ nhớ lâu dài"""
    
    # Tạo config với thread_id để theo dõi conversation
    config = {"configurable": {"thread_id": thread_id}}
    
    # Lấy state hiện tại (nếu có)
    try:
        current_state = persistent_chatbot.get_state(config)
        if current_state.values:
            # Đã có cuộc hội thoại trước đó
            state = current_state.values
            state["messages"].append(f"User: {user_input}")
            print(f"📜 Tiếp tục cuộc hội thoại (Thread: {thread_id})")
        else:
            # Cuộc hội thoại mới
            state = {
                "user_name": "Người dùng",
                "messages": [f"User: {user_input}"],
                "conversation_id": "",
                "timestamp": "",
                "context": {}
            }
            print(f"🆕 Bắt đầu cuộc hội thoại mới (Thread: {thread_id})")
    except:
        # Trường hợp không có state
        state = {
            "user_name": "Người dùng",
            "messages": [f"User: {user_input}"],
            "conversation_id": "",
            "timestamp": "",
            "context": {}
        }
        print(f"🆕 Bắt đầu cuộc hội thoại mới (Thread: {thread_id})")
    
    # Chạy workflow
    result = persistent_chatbot.invoke(state, config)
    
    # Hiển thị tin nhắn mới nhất
    latest_bot_message = [msg for msg in result["messages"] if msg.startswith("Bot:")][-1]
    print(f"🤖 {latest_bot_message}")
    
    return result

print("✅ Đã tạo hàm chat_with_memory")

In [ ]:
# Demo persistent memory
print("=== DEMO PERSISTENT MEMORY ===")
print("\n💬 Cuộc hội thoại 1:")
chat_with_memory("Xin chào, tôi tên An")

print("\n💬 Cuộc hội thoại 2 (cùng thread):")
chat_with_memory("Bạn còn nhớ tên tôi không?", "user_123")

print("\n💬 Cuộc hội thoại 3 (thread khác):")
chat_with_memory("Tôi tên Bình, bạn có nhớ tên tôi không?", "user_456")

print("\n💬 Quay lại cuộc hội thoại 1:")
chat_with_memory("Tôi thích lập trình Python", "user_123")

## Ví dụ 5: State Management Pattern nâng cao

Ví dụ về workflow phức tạp với nhiều loại state khác nhau.

In [ ]:
# State Schema nâng cao cho hệ thống đặt hàng
class OrderState(TypedDict):
    """State cho hệ thống xử lý đơn hàng"""
    order_id: str
    customer_info: dict
    items: List[dict]
    total_amount: float
    status: str  # pending, validated, processed, completed, failed
    error_messages: List[str]
    processing_steps: List[str]
    metadata: dict

def validate_order(state: OrderState) -> OrderState:
    """Node kiểm tra tính hợp lệ của đơn hàng"""
    print(f"🔍 Kiểm tra đơn hàng: {state['order_id']}")
    
    state["processing_steps"].append("Bắt đầu validation")
    
    # Kiểm tra thông tin khách hàng
    customer = state["customer_info"]
    if not customer.get("name") or not customer.get("email"):
        state["error_messages"].append("Thiếu thông tin khách hàng")
        state["status"] = "failed"
        return state
    
    # Kiểm tra items
    if not state["items"]:
        state["error_messages"].append("Đơn hàng không có sản phẩm")
        state["status"] = "failed"
        return state
    
    # Tính tổng tiền
    total = sum(item.get("price", 0) * item.get("quantity", 0) for item in state["items"])
    state["total_amount"] = total
    
    state["status"] = "validated"
    state["processing_steps"].append(f"Validation thành công - Tổng: ${total}")
    
    return state

def process_payment(state: OrderState) -> OrderState:
    """Node xử lý thanh toán"""
    print(f"💳 Xử lý thanh toán: ${state['total_amount']}")
    
    state["processing_steps"].append("Bắt đầu xử lý thanh toán")
    
    # Simulate payment processing
    if state["total_amount"] > 1000:
        state["error_messages"].append("Số tiền vượt quá giới hạn")
        state["status"] = "failed"
        return state
    
    # Payment successful
    state["status"] = "processed"
    state["processing_steps"].append("Thanh toán thành công")
    state["metadata"]["payment_date"] = datetime.now().isoformat()
    
    return state

def finalize_order(state: OrderState) -> OrderState:
    """Node hoàn tất đơn hàng"""
    print(f"✅ Hoàn tất đơn hàng: {state['order_id']}")
    
    state["processing_steps"].append("Hoàn tất đơn hàng")
    state["status"] = "completed"
    state["metadata"]["completion_date"] = datetime.now().isoformat()
    
    return state

# Điều kiện để quyết định luồng thực thi
def should_continue_processing(state: OrderState) -> str:
    """Quyết định có tiếp tục xử lý hay không"""
    if state["status"] == "failed":
        return "failed"
    elif state["status"] == "validated":
        return "process_payment"
    elif state["status"] == "processed":
        return "finalize"
    else:
        return "end"

print("✅ Đã định nghĩa Order Processing Nodes")

In [ ]:
# Tạo Order Processing Workflow
order_workflow = StateGraph(OrderState)

# Thêm nodes
order_workflow.add_node("validate", validate_order)
order_workflow.add_node("process_payment", process_payment)
order_workflow.add_node("finalize", finalize_order)

# Thêm conditional edges
order_workflow.add_edge(START, "validate")
order_workflow.add_conditional_edges(
    "validate",
    should_continue_processing,
    {
        "process_payment": "process_payment",
        "failed": END
    }
)
order_workflow.add_conditional_edges(
    "process_payment",
    should_continue_processing,
    {
        "finalize": "finalize",
        "failed": END
    }
)
order_workflow.add_edge("finalize", END)

# Compile workflow
order_processor = order_workflow.compile()

print("✅ Đã tạo Order Processing Workflow")

In [ ]:
# Test Order Processing với nhiều trường hợp
def test_order_processing():
    """Test workflow xử lý đơn hàng"""
    
    print("=== TEST ORDER PROCESSING WORKFLOW ===")
    
    # Test case 1: Đơn hàng hợp lệ
    print("\n📦 Test Case 1: Đơn hàng hợp lệ")
    valid_order = {
        "order_id": "ORD-001",
        "customer_info": {
            "name": "Nguyễn Văn A",
            "email": "a@example.com"
        },
        "items": [
            {"name": "Laptop", "price": 500, "quantity": 1},
            {"name": "Mouse", "price": 20, "quantity": 2}
        ],
        "total_amount": 0,
        "status": "pending",
        "error_messages": [],
        "processing_steps": [],
        "metadata": {}
    }
    
    result1 = order_processor.invoke(valid_order)
    print(f"Status: {result1['status']}")
    print(f"Total: ${result1['total_amount']}")
    print("Processing steps:")
    for step in result1['processing_steps']:
        print(f"  - {step}")
    
    # Test case 2: Đơn hàng thiếu thông tin
    print("\n❌ Test Case 2: Đơn hàng thiếu thông tin")
    invalid_order = {
        "order_id": "ORD-002",
        "customer_info": {"name": "Nguyễn Văn B"},  # Thiếu email
        "items": [{"name": "Phone", "price": 300, "quantity": 1}],
        "total_amount": 0,
        "status": "pending",
        "error_messages": [],
        "processing_steps": [],
        "metadata": {}
    }
    
    result2 = order_processor.invoke(invalid_order)
    print(f"Status: {result2['status']}")
    print("Errors:")
    for error in result2['error_messages']:
        print(f"  - {error}")
    
    # Test case 3: Đơn hàng vượt giới hạn
    print("\n💰 Test Case 3: Đơn hàng vượt giới hạn")
    expensive_order = {
        "order_id": "ORD-003",
        "customer_info": {
            "name": "Nguyễn Văn C",
            "email": "c@example.com"
        },
        "items": [{"name": "Server", "price": 1500, "quantity": 1}],
        "total_amount": 0,
        "status": "pending",
        "error_messages": [],
        "processing_steps": [],
        "metadata": {}
    }
    
    result3 = order_processor.invoke(expensive_order)
    print(f"Status: {result3['status']}")
    print(f"Total: ${result3['total_amount']}")
    if result3['error_messages']:
        print("Errors:")
        for error in result3['error_messages']:
            print(f"  - {error}")

# Chạy test
test_order_processing()

## Giải thích & Phân tích

### 🔍 Cách State duy trì ngữ cảnh

1. **State Persistence**: 
   - State được truyền qua tất cả các Node trong workflow
   - Mỗi Node có thể đọc và cập nhật State
   - Thay đổi được lưu trữ và chuyển đến Node tiếp theo

2. **Memory Management**:
   - `MemorySaver` lưu trữ State giữa các lần chạy
   - `thread_id` giúp phân biệt các conversation khác nhau
   - State có thể được khôi phục và tiếp tục từ điểm dừng

3. **State Schema Benefits**:
   - Định nghĩa rõ ràng cấu trúc dữ liệu
   - Type safety và validation
   - Dễ dàng mở rộng và bảo trì

### 🎯 Best Practices cho State Management

1. **Design State Schema cẩn thận**:
   - Chỉ lưu trữ thông tin cần thiết
   - Sử dụng TypedDict để type checking
   - Tách biệt data và metadata

2. **Handle State Updates đúng cách**:
   - Luôn return State sau khi cập nhật
   - Validate dữ liệu trước khi cập nhật
   - Log các thay đổi quan trọng

3. **Manage Memory hiệu quả**:
   - Sử dụng unique thread_id cho mỗi session
   - Cleanup old conversations định kỳ
   - Monitor memory usage trong production

### 📊 So sánh với các pattern khác

| Aspect | LangGraph State | Global Variables | Database | Session Storage |
|--------|----------------|------------------|----------|-----------------|
| **Scope** | Workflow-specific | Application-wide | Persistent | User session |
| **Threading** | Thread-safe | Risky | Thread-safe | Varies |
| **Persistence** | Configurable | No | Yes | Limited |
| **Type Safety** | Strong | Weak | Varies | Weak |
| **Performance** | High | High | Medium | High |
| **Scalability** | Good | Poor | Excellent | Good |

## Kết luận & Gợi ý

### 🎓 Những điều đã học

1. **State Management cơ bản**:
   - Định nghĩa State Schema với TypedDict
   - Tạo và sử dụng StateGraph
   - Đọc và cập nhật State trong Node

2. **Persistent Memory**:
   - Sử dụng MemorySaver cho lưu trữ lâu dài
   - Quản lý multiple conversations với thread_id
   - Khôi phục và tiếp tục State từ các session trước

3. **Advanced Patterns**:
   - Conditional edges dựa trên State
   - Error handling và status tracking
   - Complex workflows với multiple State types

### 🚀 Bước tiếp theo

1. **Thực hành**:
   - Tạo State Schema cho domain cụ thể của bạn
   - Thử nghiệm với different Checkpointer types
   - Implement error recovery mechanisms

2. **Nâng cao**:
   - Học về LangGraph Tools và External APIs
   - Tích hợp với real databases
   - Performance optimization cho large States

3. **Production Ready**:
   - Implement proper logging và monitoring
   - Handle concurrent access
   - Backup và recovery strategies

### 💡 Tips để thành thạo

- **Start Simple**: Bắt đầu với State Schema đơn giản, mở rộng dần
- **Test Thoroughly**: Luôn test các edge cases và error scenarios
- **Monitor Performance**: Theo dõi memory usage và execution time
- **Document Well**: Ghi chú rõ ràng về State structure và lifecycle

---

**📚 Tài liệu tham khảo:**
- [LangGraph State Documentation](https://python.langchain.com/docs/langgraph/concepts/state)
- [Checkpointer Guide](https://python.langchain.com/docs/langgraph/how-tos/persistence)
- [TypedDict Python Documentation](https://docs.python.org/3/library/typing.html#typing.TypedDict)